## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [24]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 9 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [25]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [26]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [27]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

No esta listo aun


Le echamos un vistazo para asegurar que ha ido todo bien

In [18]:
datos_row

,Unnamed: 0,Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Fecha de la última vacuna registrada (2)
0,Andalucía,140295,69445,0.494993,2021-01-07
1,Aragón,23715,9944,0.419313,2021-01-08
2,Asturias,23720,17843,0.752234,2021-01-08
3,Baleares,12840,4257,0.331542,2021-12-07
4,Canarias,27660,9572,0.346059,2021-01-08
5,Cantabria,11850,2334,0.196962,2021-01-08
6,Castilla y Leon,51390,28325,0.551177,2021-01-07
7,Castilla La Mancha,35550,12380,0.348242,2021-01-07
8,Cataluña,120545,38826,0.322087,2021-01-07
9,C. Valenciana,61225,15965,0.260759,2021-01-08


..hacemos una copia para hacerle las transformaciones necesarias:

In [ ]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [ ]:
datos_hoy.columns=['ccaa','Dosis entregadas','Dosis administradas','% sobre entregadas','Fecha de la ultima vacuna registrada']

In [ ]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [19]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [ ]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [ ]:
datos_hoy=datos_hoy[['date_pub','ccaa','Fecha de la ultima vacuna registrada','Dosis entregadas','Dosis administradas','% sobre entregadas','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [ ]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [ ]:
datos_hoy['ccaa'].unique()

In [ ]:
datos_hoy

guardamos el fichero de hoy en el directorio csv

In [ ]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [ ]:
# datos_acumulados = pd.read_csv('csv/Libro1.csv',sep=';')

In [ ]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [ ]:
datos_acumulados

concatenamos...

In [ ]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [ ]:
datos_acumulados

In [ ]:
datos_acumulados['ccaa'].unique()

y volvemos a subirlo..

In [ ]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)